Howtoloadresults.com forever and ever

In [1]:
from shared.utilities import load_results
from joblib import Parallel, delayed

In [2]:
python_results = load_results(
    "/Users/users/nastase/GitBub/Thesis/code/Data/W5_subhalo_movements", 49, 0
)

Reading your stuffy stuff from: /Users/users/nastase/GitBub/Thesis/code/Data/W5_subhalo_movements/coordinates_base_snapshot_49_subhalo_0.pickle


In [3]:
python_results

{49: array([[52521.355, 70708.15 , 41825.98 ],
        [52534.23 , 70710.01 , 41818.867],
        [52524.918, 70713.94 , 41818.19 ],
        ...,
        [52987.043, 70131.53 , 41911.6  ],
        [52526.56 , 70593.734, 41896.27 ],
        [52469.33 , 70511.664, 41839.24 ]], dtype=float32),
 135: array([[52222.676, 70832.13 , 41872.965],
        [52222.945, 70839.555, 41871.484],
        [52227.707, 70846.414, 41872.586],
        ...,
        [52303.562, 70864.73 , 41862.434],
        [52264.156, 70857.49 , 41920.594],
        [52216.188, 70750.35 , 41844.543]], dtype=float32),
 133: array([[52228.62 , 70815.984, 41897.52 ],
        [52245.105, 70832.766, 41885.11 ],
        [52238.06 , 70828.086, 41885.36 ],
        ...,
        [52176.61 , 70791.77 , 41907.14 ],
        [52177.156, 70803.13 , 41829.676],
        [52262.383, 70916.195, 41887.008]], dtype=float32),
 131: array([[52242.555, 70801.23 , 41913.168],
        [52252.15 , 70818.25 , 41904.96 ],
        [52253.27 , 70820.06 , 